In [2]:
use std::env;
use std::path::Path;

fn main() -> std::io::Result<()> {
    let path = env::current_dir()?;
    println!("The current directory is {}", path.display());
    Ok(())
}
main();

let path = Path::new("/home/bzhan/projects/postflop-solver");
assert!(env::set_current_dir(&path).is_ok());
println!("Successfully changed working directory to {}!", path.display());

The current directory is /home/bzhan/projects/postflop-solver


Successfully changed working directory to /home/bzhan/projects/postflop-solver!


In [14]:
:dep postflop-solver = { path = "/home/bzhan/projects/postflop-solver" }
use postflop_solver::*;

Error: expected value, found enum `StudAction`

In [17]:
let mut tree_config = StudTreeConfig {
    initial_state: StudBoardState::SixthStreet,
    starting_pot_from_antes: 10,
    effective_stack: 100,
    bets_cap: 4,
    small_bet_size: 2,
    big_bet_size: 4,
};

In [ ]:
let action_tree = StudActionTree::new(tree_config).unwrap();

In [9]:
use polars::prelude::*;
use std::collections::HashMap;

fn main() -> PostFlopGame { //Result<(DataFrame, DataFrame)> {
    // ranges of OOP and IP in string format
    // see the documentation of `Range` for more details about the format
    let oop_range = "66+,A8s+,A5s-A4s,AJo+,K9s+,KQo,QTs+,JTs,96s+,85s+,75s+,65s,54s";
    let ip_range = "QQ-22,AQs-A2s,ATo+,K5s+,KJo+,Q8s+,J8s+,T7s+,96s+,86s+,75s+,64s+,53s+";

    let card_config = CardConfig {
        range: [oop_range.parse().unwrap(), ip_range.parse().unwrap()],
        flop: flop_from_str("Td9d6h").unwrap(),
        turn: card_from_str("Qc").unwrap(),
        river: NOT_DEALT,
    };

    // bet sizes -> 60% of the pot, geometric size, and all-in
    // raise sizes -> 2.5x of the previous bet
    // see the documentation of `BetSizeOptions` for more details
    let bet_sizes = BetSizeOptions::try_from(("60%, e, a", "2.5x")).unwrap();

    let tree_config = TreeConfig {
        initial_state: BoardState::Turn, // must match `card_config`
        starting_pot: 200,
        effective_stack: 900,
        rake_rate: 0.0,
        rake_cap: 0.0,
        flop_bet_sizes: [bet_sizes.clone(), bet_sizes.clone()], // [OOP, IP]
        turn_bet_sizes: [bet_sizes.clone(), bet_sizes.clone()],
        river_bet_sizes: [bet_sizes.clone(), bet_sizes],
        turn_donk_sizes: None, // use default bet sizes
        river_donk_sizes: Some(DonkSizeOptions::try_from("50%").unwrap()),
        add_allin_threshold: 1.5, // add all-in if (maximum bet size) <= 1.5x pot
        force_allin_threshold: 0.15, // force all-in if (SPR after the opponent's call) <= 0.15
        merging_threshold: 0.1,
    };

    // build the game tree
    // `ActionTree` can be edited manually after construction
    let action_tree = ActionTree::new(tree_config).unwrap();
    let mut game = PostFlopGame::with_config(card_config, action_tree).unwrap();

    // obtain the private hands
    let oop_cards: &[(u8, u8)] = game.private_cards(0);
    let oop_cards_str = holes_to_strings(oop_cards).unwrap();
    println!("{:?}", oop_cards_str);

    let ip_cards = game.private_cards(1);
    let ip_cards_str = holes_to_strings(ip_cards).unwrap();
    println!("{:?}", ip_cards_str);
    
    let (mem_usage, mem_usage_compressed) = game.memory_usage();
    println!(
        "Memory usage without compression (32-bit float): {:.2}GB",
        mem_usage as f64 / (1024.0 * 1024.0 * 1024.0)
    );
    println!(
        "Memory usage with compression (16-bit integer): {:.2}GB",
        mem_usage_compressed as f64 / (1024.0 * 1024.0 * 1024.0)
    );

    game.allocate_memory(false);

    let max_num_iterations = 1000;
    let target_exploitability = game.tree_config().starting_pot as f32 * 0.005; // 0.5% of the pot
    println!("Target exploitability: {:.2}", target_exploitability);
    //let exploitability = solve(&mut game, max_num_iterations, target_exploitability, true);
    //println!("Exploitability: {:.2}", exploitability);
    
    let mut last_exploit = 1000.0;
    for i in 0..max_num_iterations {
        solve_step(&game, i);
        if (i + 1) % 100 == 0 {
            let exploitability = compute_exploitability(&game);
            println!("Exploitability on iteration {}: {:.2}", i, exploitability);
            if ((last_exploit - exploitability) < 0.01) {
                break
            }
            last_exploit=exploitability;
        }
     }
    finalize(&mut game);
    game.cache_normalized_weights();
    game
    }

In [10]:
bet_sizes

BetSizeOptions { bet: [PotRelative(0.6), Geometric(0, inf), AllIn], raise: [PrevBetRelative(2.5)] }